In [1]:
import pandas as pd
import numpy as np
import datetime
import tensorflow as tf
from tensorflow import keras
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import preprocessing

#numpy配列を省略しないようにする
np.set_printoptions(threshold=np.inf)
#pandasを省略しないようにする
pd.set_option('display.max_columns', 500) # 列
pd.set_option('display.max_rows', 500)  # 行

In [2]:
# トレーニングデータの作成
def create_data(read_dir, save_dir):
    df = pd.read_csv(read_dir, encoding='shift_jis')
    #カラムを削除
    df = df.dropna()  # NaNを削除
    df.drop(labels='日付(ローソク足)', axis=1, inplace=True) # ろうそく足データの日付
    
    df['日付'] = pd.to_datetime(df['日付'], format='%Y-%m-%d-%A %H:%M:%S')  # 日付カラムを日付型に変換
    df['日付(hour)'] = df['日付'].dt.hour  # hourをデータに追加
    df['日付(minute)'] = df['日付'].dt.minute  # minuteをデータに追加
    df['日付(weekday)'] = df['日付'].dt.dayofweek  # minuteをデータに追加
    # df['日付']カラムが [datetime64[ns]]型になっていて扱いづらいので最後に削除する
    #df.drop(labels='日付', axis=1, inplace=True)
    
    date = df['日付'] # 代入して変数を避難させる
    
    df = df.loc[:, '2':'日付(weekday)'].astype('float32')  # データ型をfloatに変換して代入
    df.insert(0, '日付', date)  # 日付を1列目に代入
    #df.to_csv(save_dir , encoding='shift_jis', index=False)
    return df

In [3]:
# n時間後の時刻とレートを計算
def UNIX_RATE_conversion(df, time, name):
    # UNIX時間に変更
    df['日付UNIX'] = pd.to_datetime(df['日付']).dt.tz_localize('Asia/Tokyo')
    df['日付UNIX'] = df['日付UNIX'].astype('int64') // 10**9
    
    df['日付UNIX' + name] = df['日付UNIX'].shift(time) # 1行上にずらして5分後の値にしている
    df['日付UNIX_計算後' + name] = df['日付UNIX' + name] - df['日付UNIX'] # -分後の値か表した変数
    df['RATE' + name] = df['bid_close'].shift(time)

    return df  # n分後のUNIX時間、レートの計算結果を反映している

In [4]:
# 変数宣言
read_dir = "./shape/GBP_JPY_X_train_data.csv"
save_dir =  "./model/GBPJPY_5M.csv"
after_5min = -1 # shift関数のスライド数 
name = '5min'

In [5]:
# トレーニングデータの作成
df = create_data(read_dir, save_dir)

# n時間後の時刻とレートを計算
df = UNIX_RATE_conversion(df, after_5min, name)

df = df[(df['日付UNIX_計算後' + name] < 320) & (270 < df['日付UNIX_計算後' + name])] # 
df.to_csv("./model/5分後予測のGBPJPYデータ.csv", encoding='shift_jis', index=False)

In [6]:
X_train = df.loc[:, '2':'日付(weekday)']  # 全行 , 列名称(始まり):列名称(終わり)
y_train = df.iloc[:,-1 ]  # 全行 , 最終列
#y_train = y_train.round(3)
print('入力数:', X_train.columns)
row_count = len(X_train)  # 行数を取得
column_count = len(X_train.columns)  # 列数を取得

入力数: Index(['2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       ...
       '2433', 'complete', 'volume', 'bid_open', 'bid_high', 'bid_low',
       'bid_close', '日付(hour)', '日付(minute)', '日付(weekday)'],
      dtype='object', length=2441)


In [7]:
def LSTM_model_create():
    n_inputs = len(X_train.columns)  # 入力数
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(50, activation='sigmoid',
                  recurrent_activation='sigmoid',
                  kernel_initializer='glorot_normal',
                   recurrent_initializer='orthogonal',
                               batch_input_shape=(None, n_inputs, 1)))
    #model.add(keras.layers.Dense(1200, activation='sigmoid'))
    #model.add(keras.layers.Dense(200, activation='linear'))
    model.add(keras.layers.Dense(50, activation='sigmoid'))
    #model.add(keras.layers.Dense(7, activation='linear'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    optimizer = keras.optimizers.Adam(lr=0.1)  # オプティマイザ
    loss_fn = keras.losses.mse  # 損失関数
    model.compile(loss=loss_fn, optimizer=optimizer, metrics=['mae'])  # コンパイル
    #model.summary()
    return model

In [8]:
model = LSTM_model_create()

In [9]:
# 正規化
def mmscaler(data):
    mm = preprocessing.MinMaxScaler()  # 正規化エンコード、デコード
    data = mm.fit_transform(data)
    return data
#print(X_train)
X_train = mmscaler(X_train)
#y_train = mmscaler(y_train)
a = pd.DataFrame(X_train)
a.to_csv('a.csv', encoding='shift_jis')

In [10]:
#row_count = len(X_train)  # 行数を取得
#column_count = len(X_train.columns)  # 列数を取得
print(column_count)
X_train = np.array(X_train).reshape(row_count, column_count, -1)  # 特徴量の形状(3次元)
print(X_train.shape)
mm = preprocessing.MinMaxScaler()  # 正規化エンコード、デコード
y_train = np.array(y_train).reshape(row_count,1)
#y_train = mmscaler(y_train)   # 正規化

print(X_train.shape)
print(y_train.shape)

2441
(752, 2441, 1)
(752, 2441, 1)
(752, 1)


In [11]:
# 学習 LSTM_GBPJPY
print('GBPJPYの学習を開始します')
history_LSTM_GBPJPY = model.fit(X_train, y_train, batch_size=1,epochs=30, validation_split=0.05,
                                            #callbacks=[tensorboard,earlystopping]
                                           )

GBPJPYの学習を開始します
Epoch 1/30
714/714 [==============================] - 213s 297ms/step - loss: 19702.5244 - mae: 140.3647 - val_loss: 19550.5762 - val_mae: 139.8234
Epoch 2/30
434/714 [=================>............] - ETA: 1:20 - loss: 19704.1019 - mae: 140.3703

KeyboardInterrupt: 